In [1]:
# import necessary packages
import pandas as pd
import os

In [2]:
# import raw data
raw_data = pd.read_csv('../raw_data/student-mat.csv')

In [3]:
# engineer the improved variable
period1 = raw_data.G1
period2 = raw_data.G2
print(type(period1), type(period2))

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [4]:
improved_list = []

for n in period1 - period2:
    if n >= 0:
        improved_list.append(0)
    else:
        improved_list.append(1)

raw_data['improved'] = improved_list

assert raw_data['improved'].max() == 1 and raw_data['improved'].min() == 0
assert len(raw_data['improved']) == 395

In [5]:
# Remove missing target variable entries
cleaned_data = raw_data[~(raw_data['G3']==0)]

assert 0 not in cleaned_data['G3'].values
assert len(cleaned_data) == (395 - len(raw_data[raw_data['G3']==0]))

In [6]:
response = cleaned_data.loc[:][['G1', 'G2', 'G3']]
dependent = cleaned_data.drop(['G1', 'G2', 'G3'], axis=1)
print(response.head())
print(dependent.head())

   G1  G2  G3
0   5   6   6
1   5   5   6
2   7   8  10
3  15  14  15
4   6  10  10
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    internet romantic  famrel  freetime  goout Dalc Walc health  \
0   ...          no       no       4         3      4    1    1      3   
1   ...         yes       no       5         3      3    1    1      3   
2   ...         yes       no       4         3      2    2    3      3   
3   ...         yes      yes       3         2      2    1    1      5   
4   ...          no       no       4         3      2    1    2      5   

  absences imp

In [7]:
class DummyEncoder:
    """Encode a DataFrame to binary categories
    
    Parameters
    ----------
    df : pandas DataFrame to encode
    
    Methods
    -------
    fit : for finding the categories to encode using
        sklearn.preprocessing.OneHotEncoder
    transform : for generating the binary representation of the 
        encoded categories and dropping the first feature of the 
        category
    fit_transform : for the execution of both the fit and transform 
        methods
        
    Returns
    -------
    a numpy.ndarray of binary values for each encoded category
    """
    
    def __init__(self, drop_first=True):
        self.num_columns = 0
        self.encoders = []
        self.drop_first = drop_first
        
    def fit(self, df):
        for i in df.columns:
            ohe = OneHotEncoder(categories='auto')
            self.num_columns =+ 1
            ohe.fit(df[i].values.reshape(-1, 1))
            self.encoders.append(ohe)
            
    def transform(self, df):
        results = []
        for i, c in enumerate(df.columns):
            results.append(self.encoders[i].\
                                transform(df[c].values.reshape(-1, 1)).\
                                toarray()[:,int(self.drop_first):])
        return np.hstack(results)
    
    def fit_transform(self, df):
        self.fit(df)
        return self.transform(df)

In [8]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [9]:
encoder = DummyEncoder()

In [10]:
encoded_data = encoder.fit_transform(dependent)

In [11]:
type(encoded_data)

numpy.ndarray

In [12]:
encoded_data.shape

(357, 108)

In [13]:
raw_data.shape

(395, 34)

In [18]:
encoded_data[:1][:]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
        0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
        0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

```python
# convert binary variables to 0 & 1
mymap = {'school': {'MS': 0, 'GP': 1},
         'sex': {'M': 0, 'F': 1},
         'address': {'U': 0, 'R': 1},
         'famsize': {'LE3': 0, 'GT3': 1},
         'Pstatus': {'A': 0, 'T': 1},
         'schoolsup': {'no': 0, 'yes': 1},
         'famsup': {'no': 0, 'yes': 1},
         'paid': {'no': 0, 'yes': 1},
         'activities': {'no': 0, 'yes': 1},
         'nursery': {'no': 0, 'yes': 1},
         'higher': {'no': 0, 'yes': 1},
         'internet': {'no': 0, 'yes': 1},
         'romantic': {'no': 0, 'yes': 1}}

for col, mapping in mymap.items():
    raw_data[col] = raw_data[col].map(mapping)

    assert raw_data[col].max() == 1 and raw_data[col].min() == 0
    assert len(raw_data[col]) == 395
```

In [14]:
# export cleaned data to directory
# cleaned_data.to_csv('cleaned-data.csv')

# assert os.path.exists('./cleaned-data.csv')